In [1]:
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from sklearn.metrics import f1_score
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.model_selection import train_test_split



pd.options.mode.chained_assignment = None

In [2]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), 
#           " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(
#         gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [4]:
torch.cuda.empty_cache()
#printm()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls '/content/drive/My Drive/app'

'Binary Model.ipynb'	       hw3-krekun.ipynb        text_binary.csv
'Binary Model SuppDev.ipynb'   MF_with_pytorch.ipynb   text.csv
 cars_test		       Multiclass.ipynb        text_multilabel.csv
 cars_train		       ratings.csv	       train
 devkit			       resized_data.csv        valid
 glove.6B.50d.txt	       test


In [7]:
mimic_data = pd.read_csv("/content/drive/My Drive/app/text_binary.csv")
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,59835582,16043746,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51487790,16456872,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59750073,16824069,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
mimic_data = mimic_data.loc[mimic_data['No Finding'] == 0]
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,50629293,16059088,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,50113713,16059088,b' WET READ: ___ ___ ___ 7:27 PM\n Left PICC ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,57620089,16059088,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
binary_data = mimic_data[['text','Support Devices']]
binary_data.text = binary_data.text.str.lower()
binary_data.head()

,text,Support Devices
0,"b"" final repor...",0.0
1,b' final repor...,0.0
5,"b"" final repor...",1.0
6,b' wet read: ___ ___ ___ 7:27 pm\n left picc ...,1.0
7,b' final repor...,1.0


In [10]:
binary_data.groupby('Support Devices').count()

,text
Support Devices,
0.0,95987
1.0,56385


In [11]:
negative = binary_data.loc[binary_data['Support Devices']==0]
print(len(negative))
positive = binary_data.loc[binary_data['Support Devices']==1].sample(n=len(negative),replace=True)

95987


In [12]:
binary_data = positive.append(negative,ignore_index=True).reset_index(drop=True)
#df1.append(df2, ignore_index=True)

In [13]:
len(binary_data)

191974

In [14]:
binary_data.text[0]

"b'                                 final report\\n history:  intubation.  evaluate et tube.  \\n \\n comparison:  none available. \\n \\n technique:  portable ap chest radiograph.\\n \\n findings:\\n \\n endotracheal tube terminates 2.8 cm above the carina.  increased density at\\n the right hila could be secondary to low lung volumes and vascular crowding.\\n otherwise, no focal consolidation, large pleural effusion or pneumothorax is\\n seen.  \\n \\n impression:\\n \\n 1. endotracheal tube terminates 2.8 cm above the carina, withdrawal of at\\n least 1 cm is recommended to ensure adequate positioning. \\n \\n 2.  nasogastric tube just reaches the ge junction, side hole is at the level\\n of the mid esophagus.  advancement is recommended.  \\n \\n 3.  decreased density in the right hilar to be secondary to low volumes and\\n reflection of bronchovascular crowding, however attention on follow up\\n recommended there is any concern for aspiration or infectious process.  \\n \\n discus

In [15]:
y = binary_data['Support Devices']
X = binary_data.drop(columns=['Support Devices'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [16]:
X_train.head()

,text
0,b' final repor...
1,b' final repor...
2,b' final repor...
3,b' final repor...
4,b' final repor...


In [17]:
# remove all '\\n' from the text
re_newlines = re.compile('\\\\n')
def sub_newlines(x): return re_newlines.sub('',x)

# remove all special characters from the text, keep only alphanumeric and spaces
re_letters = re.compile('[^A-Za-z0-9 ]')
def sub_letters(x): return re_letters.sub('', x)

# remove excessive spacing otherwise you end up with " " substrings
re_spaces = re.compile('\s+')
def sub_spaces(x): return re_spaces.sub(' ', x)
                
# tokenize all words.
my_tok = spacy.load('en')
def spacy_tok(x): 
    return [tok.text for tok in my_tok.tokenizer(sub_spaces
                                                 (sub_letters
                                                 (sub_newlines(x))))]

In [18]:
sub_spaces(sub_letters(sub_newlines(binary_data.text[0])))

'b final report history intubation evaluate et tube comparison none available technique portable ap chest radiograph findings endotracheal tube terminates 28 cm above the carina increased density at the right hila could be secondary to low lung volumes and vascular crowding otherwise no focal consolidation large pleural effusion or pneumothorax is seen impression 1 endotracheal tube terminates 28 cm above the carina withdrawal of at least 1 cm is recommended to ensure adequate positioning 2 nasogastric tube just reaches the ge junction side hole is at the level of the mid esophagus advancement is recommended 3 decreased density in the right hilar to be secondary to low volumes and reflection of bronchovascular crowding however attention on follow up recommended there is any concern for aspiration or infectious process discussed with dr by via telephone on at 1350 min after discovery'

In [19]:
#spacy_tok(binary_data.text[0])[1:]

In [20]:
binary_train = binary_data.copy()
binary_train.head()

,text,Support Devices
0,b' final repor...,1.0
1,b' final repor...,1.0
2,"b"" final repor...",1.0
3,b' wet read: ___ ___ ___ 7:03 pm\n 2 right ch...,1.0
4,b' final repor...,1.0


In [21]:
def get_counts(text):
    counts = Counter()
    for word in text:
        counts.update(spacy_tok(word)[1:])
    return counts

In [22]:
word_count = get_counts(binary_train.text)

In [23]:
len(word_count.keys())

30889

In [24]:
for word in list(word_count):
    if word_count[word] < 3:
        del word_count[word]

In [25]:
len(word_count.keys())

15944

In [26]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [27]:
len(words)

15946

In [28]:
def encode_sentence(text):
    enc = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in text.split()])
    return enc

In [29]:
encode_sentence(binary_train.text[0])

array([ 1,  2,  1,  1,  1,  6,  7,  1,  1,  1,  1, 10,  1,  1,  1,  1, 13,
       14, 15,  1,  1,  1,  1, 18,  8, 19,  1, 21, 22, 23,  1, 25, 26,  1,
       23, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,  1,  1, 41, 42,  1,
       44, 45, 46, 47, 48,  1,  1,  1,  1,  1,  1,  1, 18,  8, 19,  1, 21,
       22, 23,  1, 53, 54,  1, 55, 52, 21, 49, 56, 33, 57, 58,  1,  1,  1,
        1, 61,  8, 62, 63, 23, 64,  1, 66, 67, 49, 27, 23,  1, 54, 23, 69,
        1, 71, 49,  1,  1,  1,  1, 73, 26, 74, 23, 28, 75, 33, 31, 32, 33,
       34, 36,  1, 76, 54, 77,  1, 78, 79, 80, 81,  1, 56, 83, 49, 84, 85,
       86, 87, 47, 88,  1,  1,  1, 90, 91,  1,  1, 93,  1, 94, 95, 80,  1,
       27,  1,  1, 97, 98,  1])

## Dataset and data loaders

In [30]:
class Binary_Mimic(Dataset):
    def __init__(self, X, y, vocab):
        self.x = [encode_sentence(x) for x in X.text]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [31]:
b_train = Binary_Mimic(X_train, y_train, vocab2index)
b_valid = Binary_Mimic(X_val, y_val, vocab2index)

In [32]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [33]:
# data = [b_train[0], b_train[1], b_train[2]]
# data

In [34]:
# collate_fn(data)

In [35]:
# train_loader = DataLoader(b_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
# sents, lengths, labels = next(iter(train_loader))
# sents

In [36]:
# sents.shape, labels.shape

In [37]:
# lengths

## Training

In [38]:
def train_epocs(model, optimizer, train_dl, valid_dl, epochs=10):
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.cuda().long()
            y = y.cuda().float()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("Epoch #%.f: train loss %.3f val loss %.3f and val accuracy %.3f" % 
              (i+1,sum_loss/total, val_loss, val_acc))
  

In [39]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    with torch.no_grad():
      for x, s, y in valid_dl:
          x = x.cuda().long()
          y = y.cuda().float().unsqueeze(1)
          y_hat = model(x, s)
          loss = F.binary_cross_entropy_with_logits(y_hat, y)
          y_pred = y_hat > 0
          correct += (y_pred.float() == y).float().sum()
          total += y.shape[0]
          sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Basic GRU Model

In [40]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, lengths):
        #print(x.shape)
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [44]:
torch.cuda.empty_cache()
#printm()

In [45]:
batch_size = 25000
train_dl = DataLoader(b_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(b_valid, batch_size=batch_size, collate_fn=collate_fn)

In [46]:
vocab_size = len(words)
print(vocab_size)
model = GRUModel(vocab_size, 50, 50).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

15946


In [47]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.707 val loss 0.656 and val accuracy 0.640
Epoch #2: train loss 0.644 val loss 0.599 and val accuracy 0.673
Epoch #3: train loss 0.560 val loss 0.464 and val accuracy 0.795
Epoch #4: train loss 0.427 val loss 0.385 and val accuracy 0.859
Epoch #5: train loss 0.366 val loss 0.291 and val accuracy 0.899
Epoch #6: train loss 0.300 val loss 0.248 and val accuracy 0.917
Epoch #7: train loss 0.252 val loss 0.214 and val accuracy 0.931
Epoch #8: train loss 0.212 val loss 0.190 and val accuracy 0.939
Epoch #9: train loss 0.188 val loss 0.169 and val accuracy 0.947
Epoch #10: train loss 0.168 val loss 0.153 and val accuracy 0.952
Epoch #11: train loss 0.151 val loss 0.138 and val accuracy 0.956
Epoch #12: train loss 0.134 val loss 0.129 and val accuracy 0.958
Epoch #13: train loss 0.124 val loss 0.119 and val accuracy 0.961
Epoch #14: train loss 0.114 val loss 0.113 and val accuracy 0.964
Epoch #15: train loss 0.107 val loss 0.108 and val accuracy 0.965


## Use Glove to create pre-trained embedding 

In [48]:
def loadGloveModel(gloveFile='/content/drive/My Drive/app/glove.6B.50d.txt'):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [49]:
word_vecs = loadGloveModel()

In [50]:
word_count = get_counts(binary_train.text)

In [51]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 30889


In [52]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [53]:
word_count = delete_rare_words(word_vecs, word_count, min_df=3)
print(len(word_count.keys()))

19472


In [54]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [55]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count, min_df=3)

In [56]:
len(pretrained_weight)

19474

In [57]:
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0462,  0.1553,  0.0927,  ...,  0.1854, -0.2133, -0.2312],
        [-0.4124,  0.6493, -0.5585,  ...,  0.2621,  0.1045, -0.4430],
        ...,
        [-0.6107, -0.0947,  0.2727,  ...,  0.4242,  0.9317,  0.2807],
        [-0.1321, -0.3749, -0.5517,  ...,  0.9329,  0.3362, -0.2008],
        [ 0.8547,  0.1916,  0.0963,  ...,  0.1707, -0.2616,  0.5350]])

# GRU Model with Pretrained embedding layer:

In [58]:
class GRUModel_pre(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel_pre,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, lengths):
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [59]:
torch.cuda.empty_cache()
#printm()

In [60]:
vocab_size = len(pretrained_weight)
model = GRUModel_pre(vocab_size, 50, 50, glove_weights=pretrained_weight).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [61]:
train_epocs(model, optimizer,train_dl, valid_dl, epochs=5)

Epoch #1: train loss 0.688 val loss 0.659 and val accuracy 0.608
Epoch #2: train loss 0.642 val loss 0.533 and val accuracy 0.751
Epoch #3: train loss 0.556 val loss 0.546 and val accuracy 0.744
Epoch #4: train loss 0.485 val loss 0.393 and val accuracy 0.841
Epoch #5: train loss 0.436 val loss 0.351 and val accuracy 0.859


In [62]:
model.embeddings.weight.requires_grad = True

In [63]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.545 val loss 0.361 and val accuracy 0.874
Epoch #2: train loss 0.392 val loss 0.314 and val accuracy 0.881
Epoch #3: train loss 0.314 val loss 0.251 and val accuracy 0.913
Epoch #4: train loss 0.266 val loss 0.234 and val accuracy 0.911
Epoch #5: train loss 0.228 val loss 0.195 and val accuracy 0.932
Epoch #6: train loss 0.199 val loss 0.172 and val accuracy 0.945
Epoch #7: train loss 0.178 val loss 0.155 and val accuracy 0.948
Epoch #8: train loss 0.162 val loss 0.141 and val accuracy 0.952
Epoch #9: train loss 0.149 val loss 0.131 and val accuracy 0.956
Epoch #10: train loss 0.137 val loss 0.123 and val accuracy 0.959
Epoch #11: train loss 0.129 val loss 0.117 and val accuracy 0.962
Epoch #12: train loss 0.121 val loss 0.113 and val accuracy 0.964
Epoch #13: train loss 0.117 val loss 0.108 and val accuracy 0.965
Epoch #14: train loss 0.110 val loss 0.104 and val accuracy 0.966
Epoch #15: train loss 0.105 val loss 0.101 and val accuracy 0.968
